## work flow
* Import necessary dependencies
* Download data from Chembl a public biologicald data library
* Check the flt3 which is the name of the kinase receptor
* select based on Ic50 parameter (biologically active)
* clean the data (missing value, non value, non bulean value)
* Rearrange the data/combining the data into imporatnt columns
* Restore the data in dataframe_2

In [1]:
import pandas as pd
from chembl_webresource_client.new_client import new_client #API to collect the data

C:\Users\hp\AppData\Local\Programs\Python\Python313\Lib\site-packages\chembl_webresource_client\__init__.py:4: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  __version__ = __import__('pkg_resources').get_distribution('chembl_webresource_client').version


In [2]:
#downloading biologically active data

target = new_client.target
target_query = target.search('FLT3') #name of the receptor kinase
targets = pd.DataFrame.from_dict(target_query)
targets.head()

,cross_references,organism,pref_name,score,species_group_flag,target_chembl_id,target_components,target_type,tax_id
0,[],Mus musculus,Receptor-type tyrosine-protein kinase FLT3,18.0,False,CHEMBL2034796,"[{'accession': 'Q00342', 'component_descriptio...",SINGLE PROTEIN,10090
1,[],Homo sapiens,Tyrosine-protein kinase receptor FLT3,16.0,False,CHEMBL1974,"[{'accession': 'P36888', 'component_descriptio...",SINGLE PROTEIN,9606
2,[],Homo sapiens,VHL/FLT3,16.0,False,CHEMBL4523735,"[{'accession': 'P36888', 'component_descriptio...",PROTEIN-PROTEIN INTERACTION,9606
3,[],Homo sapiens,Protein cereblon/Tyrosine-protein kinase recep...,16.0,False,CHEMBL4630730,"[{'accession': 'P36888', 'component_descriptio...",PROTEIN-PROTEIN INTERACTION,9606
4,[],Homo sapiens,Aurora kinase B/Receptor-type tyrosine-protein...,12.0,False,CHEMBL3430908,"[{'accession': 'P36888', 'component_descriptio...",SELECTIVITY GROUP,9606


In [3]:
selected_target = targets.target_chembl_id[1]
selected_target

'CHEMBL1974'

In [4]:
# retriving biologically activity data of IC50
activity = new_client.activity
data = activity.filter(target_chembl_id=selected_target).filter(standard_type='IC50')


In [5]:
# converting to pandas dataframe

df = pd.DataFrame.from_dict(data)
df.head()

,action_type,activity_comment,activity_id,activity_properties,assay_chembl_id,assay_description,assay_type,assay_variant_accession,assay_variant_mutation,bao_endpoint,...,target_organism,target_pref_name,target_tax_id,text_value,toid,type,units,uo_units,upper_value,value
0,None,None,866063,[],CHEMBL766072,Inhibition of chimeric PDGF receptor with FLT-...,B,None,None,BAO_0000190,...,Homo sapiens,Tyrosine-protein kinase receptor FLT3,9606,None,None,IC50,uM,UO_0000065,None,0.128
1,None,None,872532,[],CHEMBL766072,Inhibition of chimeric PDGF receptor with FLT-...,B,None,None,BAO_0000190,...,Homo sapiens,Tyrosine-protein kinase receptor FLT3,9606,None,None,IC50,uM,UO_0000065,None,0.22
2,None,None,872564,[],CHEMBL766072,Inhibition of chimeric PDGF receptor with FLT-...,B,None,None,BAO_0000190,...,Homo sapiens,Tyrosine-protein kinase receptor FLT3,9606,None,None,IC50,uM,UO_0000065,None,8.79
3,None,None,879718,[],CHEMBL766072,Inhibition of chimeric PDGF receptor with FLT-...,B,None,None,BAO_0000190,...,Homo sapiens,Tyrosine-protein kinase receptor FLT3,9606,None,None,IC50,uM,UO_0000065,None,1.91
4,None,None,884645,[],CHEMBL766072,Inhibition of chimeric PDGF receptor with FLT-...,B,None,None,BAO_0000190,...,Homo sapiens,Tyrosine-protein kinase receptor FLT3,9606,None,None,IC50,uM,UO_0000065,None,30.0


In [6]:
df.shape

(5637, 46)

In [7]:
#removing rows with missing values

df2 = df[df.standard_value.notna()] #If an element is NOT a missing value, .notna() returns True for that element.
df2.shape

(5507, 46)

In [8]:
# merging the important columns that will be used in model training

columns = ['molecule_chembl_id', 'canonical_smiles', 'standard_value']
df3 = df2[columns]
df3.head

<bound method NDFrame.head of      molecule_chembl_id                                   canonical_smiles  \
0          CHEMBL330863  COc1cc2c(N3CCN(C(=O)Nc4ccc(OC(C)C)cc4)CC3)ncnc...   
1          CHEMBL124660  COc1cc2c(N3CCN(C(=O)Nc4ccc(OC(C)C)cc4)CC3)ncnc...   
2          CHEMBL126699  COc1cc2c(N3CCN(C(=O)Nc4ccc(C#N)cc4)CC3)ncnc2cc...   
3          CHEMBL445636  COc1cc2c(N3CCN(C(=O)Nc4ccc(C#N)cc4)CC3)ncnc2cc...   
4             CHEMBL941  Cc1ccc(NC(=O)c2ccc(CN3CCN(C)CC3)cc2)cc1Nc1nccc...   
...                 ...                                                ...   
5632      CHEMBL5083143  c1ccc2c(-c3coc4cc(-c5ccc(N6CCNCC6)cc5)cnc34)cc...   
5633      CHEMBL5303487  COc1cc(-c2cn[nH]c2)ccc1C(=O)Nc1ccc2c(c1)CCNCC2.Cl   
5634      CHEMBL5303487  COc1cc(-c2cn[nH]c2)ccc1C(=O)Nc1ccc2c(c1)CCNCC2.Cl   
5635      CHEMBL4475494  CC(C)(O)[C@H](F)CN1Cc2cc(NC(=O)c3cnn4cccnc34)c...   
5636      CHEMBL3903492    Cc1n[nH]c2ncc(C(=O)N3CCC[C@H]3c3ccc(Cl)cc3)cc12   

     standard_value  
0          

#Labeling the compounds as Active/Inactive/Intermediate based on their potency value(Standard_values) 
* Those < 1000 nM  = active , 
* Those > 10000 nM = inactive

In [9]:
#writing a forloop to handle the labling

bioactivity_threshold = []
for i in df3.standard_value:
    if float (i) <= 1000:
        bioactivity_threshold.append('active')
    elif float (i) >= 10000:
        bioactivity_threshold.append('inactive')
    else:
        bioactivity_threshold.append ('intermeidate')

In [10]:
#adding a bioactivity column
#name = pd.Series(clm,'name')

bioactivity_class = pd.Series(bioactivity_threshold, name='bioactivity')
df4 = pd.concat([df3, bioactivity_class], axis=1)
df4.head()

,molecule_chembl_id,canonical_smiles,standard_value,bioactivity
0,CHEMBL330863,COc1cc2c(N3CCN(C(=O)Nc4ccc(OC(C)C)cc4)CC3)ncnc...,128.0,active
1,CHEMBL124660,COc1cc2c(N3CCN(C(=O)Nc4ccc(OC(C)C)cc4)CC3)ncnc...,220.0,active
2,CHEMBL126699,COc1cc2c(N3CCN(C(=O)Nc4ccc(C#N)cc4)CC3)ncnc2cc...,8790.0,intermeidate
3,CHEMBL445636,COc1cc2c(N3CCN(C(=O)Nc4ccc(C#N)cc4)CC3)ncnc2cc...,1910.0,intermeidate
4,CHEMBL941,Cc1ccc(NC(=O)c2ccc(CN3CCN(C)CC3)cc2)cc1Nc1nccc...,30000.0,inactive


In [11]:
# Understanding the dimensions and the bioactivity of the data

df4.shape
df4.bioactivity.value_counts()

bioactivity
active          3293
inactive        1484
intermeidate     730
Name: count, dtype: int64

In [12]:
df4.to_csv('data2.csv',index=False)